# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

In [1]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [11]:
X, y = make_regression(
    n_samples=1000,
    n_features=5
)

In [12]:
X

array([[ 1.50926062, -0.3071791 , -0.62462446,  1.17223165,  0.68780243],
       [ 0.01302596,  0.63695528, -0.31255844,  0.23513715, -0.08029111],
       [-0.74428816,  0.63408584,  1.357043  , -1.06089217,  0.29487314],
       ...,
       [ 0.1241297 , -0.7287295 , -0.61478318, -0.73344632, -0.05704723],
       [ 0.53876636,  1.95111621,  1.48125606,  1.01269738, -0.42944794],
       [ 0.24001897, -0.8161537 ,  0.09435057,  0.21486795, -0.14584053]])

In [ ]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("poly", PolynomialFeatures()),
        ("lr", LinearRegression()),
    ]
).fit(X, y)

In [ ]:
pipe.predict(X)

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
a = [1, 2, 3]
a[2] # a.__getitem__(2)
len(a)

In [ ]:
class SyntDataset(Dataset):
  def __init__(self, transform: callable = None, **make_regression_args):
    self.X, self.y = make_regression(**make_regression_args)
    # self.X = th.Tensor(self.X)
    self.transform = transform

  def __getitem__(self, idx):
    # return {
    #     "x": self.X[idx],
    #     "y": self.y[idx]
    # }
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)
    return x, self.y[idx]

  def __len__(self):
    return len(self.X)

In [ ]:
import numpy as np
def add_squares(x):
  return np.c_[x, x ** 2]

In [ ]:
x = np.random.randint(0, 200, (20, 5))
x.shape

(20, 5)

In [ ]:
add_squares(x).shape

(40, 5)

In [ ]:
class SquareN:
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    for _ in range(self.n):
      x = add_squares(x)
    return x

In [ ]:
o = SquareN(2)
# o(d.X[:4])
# add_squares(d.X[:4])

In [ ]:
# d = SyntDataset(n_samples=1000, n_features=5)
d = SyntDataset(
    transform=SquareN(2),
    n_samples=1000,
    n_features=5
)
d[0:5]

In [ ]:
d[:5]

In [ ]:
dl = DataLoader(d, batch_size=32)

In [ ]:
for (batch_X, batch_y) in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([32, 20]) torch.Size([32])


In [ ]:
def clip(x):
  return np.clip(x, 0, 1)

def add_squaresN(x, n):
  for _ in range(n):
    x = add_squares(x)
  return x

In [ ]:
x = np.random.randint(0, 200, (20, 5))
# x = clip(x)
# x = add_squaresN(x, 2)

for f in [clip, SquareN(2)]:
  x = f(x)

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [382]:
import pandas as pd

In [383]:
df = pd.read_csv('bank-full.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [384]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [385]:
pipe = Pipeline(
    [
        ("enc", OrdinalEncoder()),
        ("scaler", MinMaxScaler())
    ]
)

In [386]:
df_encoded = pipe.fit_transform(df)
pd.DataFrame(df_encoded, columns=df.columns).sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
16195,0.223684,0.636364,0.0,0.333333,0.0,0.203572,1.0,0.0,0.0,0.700000,0.454545,0.396947,0.021277,0.000000,0.00,1.0,0.0
25662,0.381579,0.363636,0.5,1.000000,0.0,0.651319,1.0,0.0,0.0,0.600000,0.818182,0.311069,0.042553,0.000000,0.00,1.0,1.0
45169,0.171053,0.363636,0.5,0.666667,0.0,0.530208,1.0,0.0,0.0,0.300000,0.818182,0.115776,0.000000,0.405018,0.05,0.0,0.0
89,0.513158,0.454545,0.5,0.333333,0.0,0.195619,1.0,0.0,1.0,0.133333,0.727273,0.114504,0.021277,0.000000,0.00,1.0,0.0
29905,0.447368,0.090909,0.0,0.000000,0.0,0.201479,0.0,0.0,0.0,0.100000,0.272727,0.195293,0.000000,0.000000,0.00,1.0,0.0


<p class="task" id="2"></p>

2\. Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [378]:
from torch.utils.data import Dataset

In [379]:
class BankDatasetBase(
    Dataset
):
    def __init__(self, data: pd.DataFrame) -> None:
        self.X, self.y = data.iloc[:,:-1], df.iloc[:,-1]
    def __getitem__(self, idx) -> tuple:
        x = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return x, y

    def __len__(self) -> int:
        return self.X.shape[0]

In [380]:
df2 = BankDatasetBase(df)
df2[5]

(age                  35
 job          management
 marital         married
 education      tertiary
 default              no
 balance             231
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            139
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 5, dtype: object,
 'no')

In [381]:
len(df2)

45211

<p class="task" id="3"></p>

3\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [375]:
class BankDataset(
    Dataset
):
    def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
        self.X, self.y = data.iloc[:,:-1], pd.DataFrame(df.iloc[:,-1])
        self.transform = transform

    def __getitem__(self, idx) -> tuple:
        # x - набор признаков из idx-й строки
        # y - набор признаков из idx-й строки
        # если при создании был передан transform
        # x, y = transform(x, y)

        x = self.X.iloc[idx]
        y = self.y.iloc[idx]

        if self.transform is not None:
          x, y = self.transform(x, y)

        return x, y

    def __len__(self) -> int:
        return self.X.shape[0]

In [376]:
import numpy as np
def test_transform(X, y):
  if np.ndim(y) < 2:
    y = np.array([y])
  if np.ndim(X) < 2:
    X = np.array([X])
  return OrdinalEncoder().fit_transform(X), OrdinalEncoder().fit_transform(y)

In [377]:
df3 = BankDataset(df, test_transform)
df3[:5]

(array([[3., 2., 0., 1., 0., 4., 1., 0., 0., 0., 0., 4., 0., 0., 0., 0.],
        [1., 3., 1., 0., 0., 2., 1., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 0., 0., 2., 0., 3., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 4., 1., 2., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [368]:
class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns

    def __call__(self, x, y) -> tuple:
      res_xy = x.copy()
      target_name = y.columns[0]
      res_xy[target_name] = y

      for name in self.category_columns:
        column = res_xy[name]
        uni_items = column.unique()

        to_nums = dict(zip(uni_items, range(len(uni_items))))
        res_xy[name] = column.map(to_nums)

      # to_nums_y = dict(zip(y.unique(),range(len(y.unique()))))
      # res_y = y.map(to_nums_y)

      return res_xy.drop(target_name,axis=1), res_xy[target_name]

In [369]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [370]:
BankDataset(df, LabelEncoderTransform(['job','marital','y']))[:5]

(   age  job  marital  education default  balance housing loan  contact  day  \
 0   58    0        0   tertiary      no     2143     yes   no  unknown    5   
 1   44    1        1  secondary      no       29     yes   no  unknown    5   
 2   33    2        0  secondary      no        2     yes  yes  unknown    5   
 3   47    3        0    unknown      no     1506     yes   no  unknown    5   
 4   33    4        1    unknown      no        1      no   no  unknown    5   
 
   month  duration  campaign  pdays  previous poutcome  
 0   may       261         1     -1         0  unknown  
 1   may       151         1     -1         0  unknown  
 2   may        76         1     -1         0  unknown  
 3   may        92         1     -1         0  unknown  
 4   may       198         1     -1         0  unknown  ,
 0    0
 1    0
 2    0
 3    0
 4    0
 Name: y, dtype: int64)

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [371]:
import torch as th

In [372]:
class ToTensor(object):
    def __call__(self, X, y) -> tuple:
      if isinstance(X, pd.DataFrame):
        X = X.values
      if isinstance(y, pd.Series) or isinstance(y, pd.DataFrame):
        y = y.values
      return th.tensor(X), th.tensor(y)

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms

    def __call__(self, X, y):
        for transform in self.transforms:
          X, y = transform(X, y)
        return X, y

In [373]:
set(df.columns) - set(df.describe().columns)

{'contact',
 'default',
 'education',
 'housing',
 'job',
 'loan',
 'marital',
 'month',
 'poutcome',
 'y'}

In [374]:
numeric_columns = set(df.columns) - set(df.describe().columns)
comp = Compose(
    [
        LabelEncoderTransform(numeric_columns),
        ToTensor()
    ]
)

BankDataset(df, comp)[:5]

(tensor([[  58,    0,    0,    0,    0, 2143,    0,    0,    0,    5,    0,  261,
             1,   -1,    0,    0],
         [  44,    1,    1,    1,    0,   29,    0,    0,    0,    5,    0,  151,
             1,   -1,    0,    0],
         [  33,    2,    0,    1,    0,    2,    0,    1,    0,    5,    0,   76,
             1,   -1,    0,    0],
         [  47,    3,    0,    2,    0, 1506,    0,    0,    0,    5,    0,   92,
             1,   -1,    0,    0],
         [  33,    4,    1,    2,    0,    1,    1,    0,    0,    5,    0,  198,
             1,   -1,    0,    0]]),
 tensor([0, 0, 0, 0, 0]))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

In [362]:
X6, y6 = BankDataset(df, comp)[:]
X6

tensor([[ 58,   0,   0,  ...,  -1,   0,   0],
        [ 44,   1,   1,  ...,  -1,   0,   0],
        [ 33,   2,   0,  ...,  -1,   0,   0],
        ...,
        [ 72,   5,   0,  ..., 184,   3,   3],
        [ 57,   3,   0,  ...,  -1,   0,   0],
        [ 37,   2,   0,  ..., 188,  11,   2]])

In [363]:
y6

tensor([0, 0, 0,  ..., 1, 0, 0])

In [364]:
n = X6.shape[0]
id_split = int(n*0.75)
X_train, X_test, y_train, y_test = X6[:id_split], X6[id_split:], y6[:id_split], y6[id_split:]
n == X_train.shape[0] + X_test.shape[0] == y_train.shape[0] + y_test.shape[0]

True

In [365]:
from torch.utils.data import DataLoader

In [366]:
dl = DataLoader(X_train.reshape(-1,2,8), batch_size=64, shuffle=True)
for i in dl:
  batch = i
  break
batch.shape

torch.Size([64, 2, 8])

In [367]:
batch

tensor([[[  39,    3,    0,  ..., 7685,    0,    0],
         [   1,   14,   10,  ...,   -1,    0,    0]],

        [[  36,    3,    0,  ...,  655,    0,    0],
         [   0,   23,    0,  ...,   -1,    0,    0]],

        [[  38,    1,    1,  ...,   23,    1,    0],
         [   1,   23,    2,  ...,   -1,    0,    0]],

        ...,

        [[  30,    0,    1,  ..., 2544,    1,    0],
         [   1,    9,   10,  ...,   -1,    0,    0]],

        [[  42,    6,    0,  ...,  101,    0,    0],
         [   0,    6,    0,  ...,   -1,    0,    0]],

        [[  47,    6,    0,  ..., 1060,    0,    0],
         [   0,    6,    1,  ...,   -1,    0,    0]]])

## Обратная связь
- [ ] Хочу получить обратную связь по решению